In [13]:
# HIDDEN
#IMPORT LIBRARIES
import os     
os.environ["PATH"] += os.pathsep + 'C:\\Anaconda3\\Library\\bin\\graphviz'
import pandas_datareader as pd2
import pandas as pd
import matplotlib.pyplot as mp
import tensorflow
import keras as ks
from keras.layers import Dense,LSTM,Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
import numpy as np
from ipywidgets import interact,interact_manual
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [5]:
# HIDDEN
#WE DECLARE OUR CLASSES
#THIS FUNCTION GETS, WINDOWS AND SPLITS INTO TEST/TRAIN
class Data():
    
    def __init__(self,ticker,start,end):
        self.data = pd2.DataReader(ticker,'yahoo', start, end).iloc[:,]
        
    def window(self,windows,col):
        
        #we sort the windowed array
        windows = sorted(windows)

        #we loop through the training data and restructure
        for i in range(max(windows),self.data.count()[0]):

            y_record = self.data.iloc[[i],:].loc[:,[col]]
            x_record = self.data.iloc[[i-windows[0]],:].loc[:,[col]]
            x_record.columns = str(windows[0]) + "-" + x_record.columns

            if(len(windows)>1):

                for j in range(1,len(windows)):
                    x_temp   = self.data.iloc[[i-windows[j]],:].loc[:,[col]]
                    x_temp.columns = str(windows[j]) + "-" + x_temp.columns
                    x_record = pd.concat([x_record.reset_index(drop=True), x_temp.reset_index(drop=True)], axis=1)

            if(i==max(windows)):
                x_final = x_record
                y_final = y_record
            else:
                x_final = x_final.append(x_record)
                y_final = y_final.append(y_record)

            self.features = x_final.set_index(y_final.index)
            self.response = y_final
            
    def split_data(self,period):
        self.test_features = self.features.iloc[self.features.count()[0]-period:self.features.count()[0],:]
        self.train_features = self.features.iloc[0:self.features.count()[0]-period,:]
        self.test_response = self.response.iloc[self.features.count()[0]-period:self.features.count()[0],:]
        self.train_response = self.response.iloc[0:self.features.count()[0]-period,:]
        self.data_train = self.data.iloc[0:self.data.count()[0]-period,:]
        self.data_test =  self.data.iloc[self.data.count()[0]-period:self.data.count()[0],:]
class NN():
    
    def __init__(self,neurons,activations):
        self.neurons = neurons
        self.activations = activations
     
    def create_model(self,dim,loss,optimizer):
    
        self.model = ks.Sequential()
     
        for i in range(0,len(self.neurons)):
            neuron = self.neurons[i]
            activation = self.activations[i]
        
            if(i == 0):
                self.model.add(Dense(neuron,input_dim = dim,activation = activation))
            else:
                self.model.add(Dense(neuron,activation = activation))
                
        self.model.compile(loss=loss,optimizer=optimizer)

Please select build your network

In [6]:
test = Data('GM','2005-05-05','2015-05-05')
test.window(windows = [30,60],col = 'Open')
test.split_data(300)

In [7]:
#we get our variables
X = test.train_features
Y = test.train_response.loc[:,'Open']

#we specify the stop criteria
Stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=50, verbose=1, mode='auto', baseline=None)

In [8]:
test_model = NN([100,100,10,1],['sigmoid','sigmoid','linear','linear'])
test_model.create_model(dim=X.shape[1],loss='mse',optimizer='adam')

hist = test_model.model.fit(X,Y,epochs=300,batch_size=2,verbose=0,callbacks=[Stop],validation_data = (test.test_features,test.test_response.loc[:,'Open']))


Epoch 00052: early stopping


In [14]:

def storage(lay,act):
    global layers
    global activations
    
    layers = lay
    activations =act

def NN_builder():
    global layers
    global activation

    n = list(map(int,layers.split(",")))
    a = activations.split(",")
    test_model = NN(n,a)
    test_model.create_model(dim=X.shape[1],loss='mse',optimizer='adam')
    print(test_model.model.summary())

interact(storage,lay="",act='')    
interact_manual(NN_builder)
    

interactive(children=(Text(value='', description='lay'), Text(value='', description='act'), Output()), _dom_cl…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.NN_builder()>

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.